In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
df_meta = pd.read_csv('/opt/NAS3/EmoryCXRv1/Tables/emory_cxr_v1_metadata_DEID.csv')
df_finding = pd.read_csv('/home/fli40/Data/Datathon2023_CXR/FINAL_FINDINGS_ANON.csv')

In [ ]:
#df_meta.columns

In [ ]:
#df_finding.columns

In [ ]:
df_finding.replace(np.nan, 0, inplace=True)
df_finding.replace(-1, np.nan, inplace=True)
df_finding.dropna(inplace=True)
df_finding

In [ ]:
PATHOLOGIES = ['Support Devices','Pneumothorax', 'No Finding']

In [ ]:
df = df_meta[['AccessionNumber_anon', 'empi_anon', 'image_path', 'sex', 'RACE', 'age','ImageView_Frontal']].merge(df_finding[['AccessionNumber_anon']+PATHOLOGIES], on='AccessionNumber_anon')
df.drop_duplicates(inplace=True)
df

In [ ]:
## select only one image for one subject

df = df[df['ImageView_Frontal']==1]
df.drop_duplicates(subset=['empi_anon'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.rename(columns={'image_path':'Path', 'sex':'Sex', 'RACE':'Race', 'age':'Age'}, inplace=True)

In [ ]:
race_map = {'Caucasian or White':'White', 
            'African American  or Black':'Black', 
            'Asian':'Asian',
            'Unknown, Unavailable or Unreported':'Other', 
            'Multiple':'Other',
            'Patient Declines':'Other', 
            'Hispanic':'Hispanic',
            'Native Hawaiian or Other Pacific Islander':'Other', 
            'Not Recorded':'Other',
            'American Indian or Alaskan Native':'Other', 
            'Asian American':'Asian',
            'Alaskan Native':'Asian'}

In [ ]:
df['Race'] = df.Race.replace(race_map)

In [ ]:
df = df[(df.Race=='Black') | (df.Race=='White')]

In [ ]:
df['Age_group'] = pd.cut(df['Age'], 
                           bins=[0, 60, 130],
                           labels=['18-60', '60+'])

In [ ]:
df

In [ ]:
#df.groupby(['Sex', 'Race', 'Age_group', 'Support Devices', 'No Finding'])['Pneumothorax'].value_counts()
df.groupby(['Sex', 'Race', 'Support Devices', 'No Finding'])['Pneumothorax'].value_counts()

In [ ]:
df.groupby(['Support Devices', 'No Finding'])['Pneumothorax'].value_counts()

In [ ]:
def combine_categories(df, cat):
    col_list = []
    for c in cat:
        col_list.append(df[c].astype(str))
        
    combined = col_list[0]
    for dfc in col_list[1:]:
        combined = combined + '_' + dfc 
    return combined

In [ ]:
df_no_finding = df[(df['No Finding'] == 1.0) & (df['Support Devices'] == 0.0)]
df_PTX = df[(df.Pneumothorax == 1.0) & (df['Support Devices'] == 1.0)]

In [ ]:
df_no_finding['group'] = combine_categories(df_no_finding, ['Sex','Race'])
df_PTX['group'] = combine_categories(df_PTX, ['Sex','Race'])

In [ ]:
df_no_finding.group.value_counts()

In [ ]:
df_PTX.group.value_counts()

In [ ]:
df_PTX_sampled = df_PTX.groupby('group').sample(n=282, replace=False)
df_PTX_sampled

In [ ]:
df_no_finding_sampled = df_no_finding.groupby('group').sample(n=282)
df_no_finding_sampled

In [ ]:
df_sampled = pd.concat([df_PTX_sampled, df_no_finding_sampled], axis=0)

In [ ]:
df_sampled.Race.value_counts()

In [ ]:
df_sampled.Sex.value_counts()

In [ ]:
df_sampled.Age_group.value_counts()

In [ ]:
df_sampled.Pneumothorax.value_counts()

In [ ]:
df_sampled['Support Devices'].value_counts()

In [ ]:
df_sampled['No Finding'].value_counts()

In [ ]:
df_sampled['Path'] = ['/'.join(l[5:]) for l in df_sampled.Path.str.split('/')]
df_sampled  

In [ ]:
df_sampled.to_csv('Data1.csv', index=False)